# Preliminaries

## Imports

In [ ]:
import tempfile
from pathlib import Path

import holoviews as hv
import panel as pn

hv.extension("bokeh")
pn.extension()
# If in google colab, run hack that allows holoviews to work properly
try:
    import google.colab  # noqa

    def _render(self, **kwargs):
        hv.extension("bokeh")
        return hv.Store.render(self)

    hv.core.Dimensioned._repr_mimebundle_ = _render
except ModuleNotFoundError:
    pass

TMP_NOTEBOOK_ROOT = Path(tempfile.mkdtemp()) / "basics" / "sample_api"

## Loading a dataset

To create Dataset objects, it's recommended to utilize a **DatasetProvider**. In this instance, we'll employ the Coco2017Detection provider:

In [ ]:
from bridge.providers.vision import Coco2017Detection

root_dir = TMP_NOTEBOOK_ROOT / "coco"

provider = Coco2017Detection(root_dir)
ds = provider.build_dataset()
ds


# Sample API

In BridgeDS, we use two complementing approaches to view datasets. We call them the **Sample API** and the **Table API**. This tutorial is about the former.

Sample API can be loosely described as:

    A dataset can be viewed as a collection of samples, where each sample is a pythonic object (called Sample) that contains a collection of elements.

In case any of the terms 'dataset', 'sample' or 'element' is foreign to you, we recommend you to first go back to the Key Concepts section.

Let's demonstrate how to use the **Sample API**:

## Indexing

`ds.iget / ds.get` are our equivalents of `df.iloc / df.loc`, used for fetching individual samples from the dataset:

In [ ]:
sample = ds.get(34)  # get sample with index 34
print("Sample ID:", sample.id)
sample = ds.iget(1)  # get sample with positional index 1
print("Sample ID:", sample.id)

## Properties
The sample object is fairly minimal, exposing only its _id_, _elements_, and _display_engine_ properties, with limited methods available. This design reflects its role as a container for elements rather than a data object itself.

Now, let’s shift our focus to the elements:

As a reminder, in BridgeDS, _elements_ can be any type of data—ranging from raw objects like images or text, to annotations like bounding boxes (bboxes), segmentation maps (segmaps), or class labels.

Let’s take a look at the elements contained in our current sample:

In [ ]:
print("Sample ID:", sample.id)
print("Total num elements in sample:", len(sample), "\n")
for etype, elist in sample.elements.items():
    print(f"Num elements with etype={etype}:", len(elist))

We observe one image element and two bbox elements. It's common to have samples compose of a single element representing raw data (the image) alongside multiple elements representing annotations. To accommodate this frequent use case, we implement COCO samples using a subclass of **Sample** called **SingularSample**. This subclass provides a more convenient API, where the main element is accessible via `sample.element`, and the remaining elements are organized under `sample.annotations`:

In [ ]:
print("The 'sample element' (the image):")
print(f"class: {type(sample).__name__}")
print(f"etype: {sample.element.etype}")
print(f"image shape: {sample.element.data.shape}")
print(f"element_id: {sample.element.id}\n")
print("The annotation elements:")
print(f"n_bboxes: {len(sample.annotations['bbox'])}")
[print(bb_element.data) for bb_element in sample.annotations["bbox"]];

As you can see, **elements** are container objects of the actual data - they have the `.data` property. A **sample** is just a collection of **elements**, and is the representation of an individual example from the dataset.

## The DisplayEngine
We will elaborate on how the DisplayEngine works in a separate tutorial, but for basic usage it's worth noting that both the Dataset and the Sample objects expose a `.show()` method, which takes advantage of the DisplayEngine and produces the following:

In [ ]:
sample.show()

In [ ]:
# display entire dataset with interactive interface:
ds.show()

As you can see, our class labels are represented as integers rather than strings. This is because the raw COCO dataset stores class labels in this format. If you'd like to learn how to convert these integers into readable strings, or explore how to perform operations across the entire dataset, check out our next tutorial on the **Table API**.